In [ ]:
import os
import numpy as np
import random
import copy
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

def getTrainTest(label, trainNum):
    random.seed(32)
    testGt = copy.deepcopy(label)
    trainGt = np.zeros(label.shape)
    labelNum = int(max(label.ravel())) + 1
    for lab in range(1, labelNum):
        labIndex = np.where(label == lab)
        randomList = random.sample(range(0, len(labIndex[0])), int(len(labIndex[0])*trainNum))
        for randInt in randomList:
            x = labIndex[0][randInt]
            y = labIndex[1][randInt]
            trainGt[x][y] = lab
            testGt[x][y] = 0
    trainGt = trainGt.astype(int)
    testGt = testGt.astype(int)
    return trainGt, testGt

datasets = ['t32ulu']
ratios = [0.5]

for dataset in datasets:
    for ratio in ratios:
        select_gt = np.load(f'/home/aistudio/npys/{dataset}/select_gt.npy').astype(int)
        select_rs = np.load(f'/home/aistudio/npys/{dataset}/select_rs.npy')[:,:,:7]

        train_gt, test_gt = getTrainTest(select_gt, ratio)

        H, W, C, T = select_rs.shape
        data = select_rs.reshape((-1, C*T))
        train_labels = train_gt.flatten()
        test_labels = test_gt.flatten()

        train_data = data[train_labels != 0, :]
        train_labels = train_labels[train_labels != 0] - 1
        test_data = data[test_labels != 0, :]
        test_labels = test_labels[test_labels != 0] - 1

        model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
        model_mlp.fit(train_data, train_labels)

        pred_labels_mlp = model_mlp.predict(test_data)
        oa_mlp = accuracy_score(test_labels, pred_labels_mlp)
        kappa_mlp = cohen_kappa_score(test_labels, pred_labels_mlp)
        cm_mlp = confusion_matrix(test_labels, pred_labels_mlp)
        class_accuracy_mlp = cm_mlp.diagonal() / cm_mlp.sum(axis=1)
        aa_mlp = np.mean(class_accuracy_mlp)
        miou_mlp = np.mean(np.diag(cm_mlp) / (cm_mlp.sum(axis=1) + cm_mlp.sum(axis=0) - np.diag(cm_mlp)))

        output_path = f'/home/aistudio/MLP/{dataset}/{ratio}'
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        with open(f'{output_path}/evaluation_metric_mlp.txt', 'w', encoding='utf-8') as f:
            f.write(f"整体精度 (OA): {oa_mlp}\n")
            f.write(f"平均交并比 (MIoU): {miou_mlp}\n")
            f.write("训练参数：\n")
            f.write(str(model_mlp.get_params()))

        print(f"完成数据集 {dataset} 与比例 {ratio} 的MLP处理。")

In [1]:
pip install xgboost

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import numpy as np
import random
import copy
import xgboost as xgb
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

def getTrainTest(label, trainNum):
    random.seed(32)
    testGt = copy.deepcopy(label)
    trainGt = np.zeros(label.shape)
    labelNum = int(max(label.ravel())) + 1
    for lab in range(1, labelNum):
        labIndex = np.where(label == lab)
        randomList = random.sample(range(0, len(labIndex[0])), int(len(labIndex[0])*trainNum))
        for randInt in randomList:
            x = labIndex[0][randInt]
            y = labIndex[1][randInt]
            trainGt[x][y] = lab
            testGt[x][y] = 0
    trainGt = trainGt.astype(int)
    testGt = testGt.astype(int)
    return trainGt, testGt

datasets = ['t31tfj']
ratios = [0.5]

for dataset in datasets:
    for ratio in ratios:
        select_gt = np.load(f'/home/aistudio/npys/{dataset}/select_gt.npy').astype(int)
        select_rs = np.load(f'/home/aistudio/npys/{dataset}/select_rs.npy')[:,:,:5]

        train_gt, test_gt = getTrainTest(select_gt, ratio)

        H, W, C, T = select_rs.shape
        data = select_rs.reshape((-1, C*T))
        train_labels = train_gt.flatten()
        test_labels = test_gt.flatten()

        train_data = data[train_labels != 0, :]
        train_labels = train_labels[train_labels != 0] - 1
        test_data = data[test_labels != 0, :]
        test_labels = test_labels[test_labels != 0] - 1

        model_xgb = xgb.XGBClassifier(n_estimators=1, use_label_encoder=False, eval_metric='mlogloss')
        model_xgb.fit(train_data, train_labels)

        pred_labels_xgb = model_xgb.predict(test_data)
        oa_xgb = accuracy_score(test_labels, pred_labels_xgb)
        kappa_xgb = cohen_kappa_score(test_labels, pred_labels_xgb)
        cm_xgb = confusion_matrix(test_labels, pred_labels_xgb)
        class_accuracy_xgb = cm_xgb.diagonal() / cm_xgb.sum(axis=1)
        aa_xgb = np.mean(class_accuracy_xgb)
        miou_xgb = np.mean(np.diag(cm_xgb) / (cm_xgb.sum(axis=1) + cm_xgb.sum(axis=0) - np.diag(cm_xgb)))

        output_path = f'/home/aistudio/XGB/{dataset}/{ratio}'
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        with open(f'{output_path}/evaluation_metric_xgb.txt', 'w', encoding='utf-8') as f:
            f.write(f"整体精度 (OA): {oa_xgb}\n")
            f.write(f"平均交并比 (MIoU): {miou_xgb}\n")
            f.write("训练参数：\n")
            f.write(str(model_xgb.get_params()))

        print(f"完成数据集 {dataset} 与比例 {ratio} 的处理。")

完成数据集 t31tfj 与比例 0.5 的处理。
